---
format:
  html:
    code-line-numbers: true
    code-overflow: wrap
    code-block-bg: true
    code-block-border-left: true
    highlight-style: Arrow
---

# The Cutting Stock Problem {#sec-cutting-stock}

The cutting stock problem is a common problem in the paper industry, where large rolls of paper must be cut into smaller rolls of various widths to meet customer demands. For example, a paper mill may have a large roll of paper that is 60 inches wide and needs to produce smaller rolls of widths 30, 24, 18, and 12 inches. The mill must determine the best way to cut the large roll of paper into the required widths while minimizing waste.

Let's say that a paper company produces rolls with a uniform width of 100 inches and receives orders for rolls with widths of 20, 30, 40, and 50 inches. 
@tbl-cutting-stock-data shows an illustrative example.


In [4]:
# | echo: false
# | label: tbl-cutting-stock-data
# | tbl-cap: Demands of different paper rolls

from IPython.display import Markdown
from tabulate import tabulate

table_data = []

row_data = []
row_data.append(20)
row_data.append(100)
table_data.append(row_data)

row_data = []
row_data.append(30)
row_data.append(120)
table_data.append(row_data)

row_data = []
row_data.append(40)
row_data.append(40)
table_data.append(row_data)

row_data = []
row_data.append(50)
row_data.append(20)
table_data.append(row_data)

col_names = ["Order Width", "Order Quantity"]
Markdown(tabulate(table_data, headers=col_names))

  Order Width    Order Quantity
-------------  ----------------
           20               100
           30               120
           40                40
           50                20

To fulfill these orders, a single 100 inch roll can be cut into one or more of the order widths. This process generates scrap, which is the leftover material that cannot be used. The different combinations of cuts are called patterns. In this case, there are 37 different patterns that can be used.

In [16]:
# | echo: false
# | label: tbl-cutting-stock-available-patterns
# | tbl-cap: Available patterns

from IPython.display import Markdown
from tabulate import tabulate

table_data = []

row_data = []
row_data.append("Pattern 1")
row_data.append(1)
row_data.append(1)
row_data.append(1)
row_data.append(0)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 2")
row_data.append(1)
row_data.append(1)
row_data.append(0)
row_data.append(1)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 3")
row_data.append(0)
row_data.append(0)
row_data.append(1)
row_data.append(1)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 4")
row_data.append(3)
row_data.append(0)
row_data.append(1)
row_data.append(0)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 5")
row_data.append(0)
row_data.append(2)
row_data.append(1)
row_data.append(0)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 6")
row_data.append(0)
row_data.append(0)
row_data.append(0)
row_data.append(2)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 7")
row_data.append(5)
row_data.append(0)
row_data.append(0)
row_data.append(0)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 8")
row_data.append(0)
row_data.append(3)
row_data.append(0)
row_data.append(0)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 9")
row_data.append(0)
row_data.append(0)
row_data.append(2)
row_data.append(0)
table_data.append(row_data)

row_data = []
row_data.append("Pattern 10")
row_data.append(2)
row_data.append(0)
row_data.append(0)
row_data.append(1)
table_data.append(row_data)

col_names = ["Pattern", "Width (20)", "Wdith (30)", "Width (40)", "Width (50)"]
Markdown(tabulate(table_data, headers=col_names))

Pattern       Width (20)    Wdith (30)    Width (40)    Width (50)
----------  ------------  ------------  ------------  ------------
Pattern 1              1             1             1             0
Pattern 2              1             1             0             1
Pattern 3              0             0             1             1
Pattern 4              3             0             1             0
Pattern 5              0             2             1             0
Pattern 6              0             0             0             2
Pattern 7              5             0             0             0
Pattern 8              0             3             0             0
Pattern 9              0             0             2             0
Pattern 10             2             0             0             1